# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import torch
from pinn import PINN
import utils

In [ ]:
in_dir = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

out_dir = "/Users/ggito/repos/pinns/data/"
models_out_dir = "/Users/ggito/repos/pinns/data/models/"
log_filepath = out_dir + "log/log.csv"

In [ ]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
elif torch.cuda.is_available():
  device = torch.device("cuda")
else:
  print("GPU device not found.")

print(device)

In [ ]:
wing_df = pd.read_csv(in_dir + points_filename)
# norm_df = pd.read_csv(in_dir + norms_filename)

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [2058, 2058, 2058]
hidden_units = [1024, 1024, 1024]

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1)

epochs = 300

Nf = 20000   # num of collocation points -> pde evaluation
N0 = 20000   # num of points to evaluate initial conditons
Nb = 20000   # num of points to evaluate boundary conditions
Nw = 20000   # num of points of the surface of the front wing to evaluate boundary conditions

# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1
mu = 1

# m/s
in_velocity = 1

# Domain limits
x_max = 1
y_max = 1
z_max = 1
t_max = 2

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [2058, 2058, 2058]
hidden_units = [1024, 1024, 1024]

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1)

epochs = 300

Nf = 20000   # num of collocation points -> pde evaluation
N0 = 20000   # num of points to evaluate initial conditons
Nb = 20000   # num of points to evaluate boundary conditions
Nw = 20000   # num of points of the surface of the front wing to evaluate boundary conditions

c1 = 1
c2 = 1
c3 = 1
c4 = 1

In [9]:
epoch_loss = []
checkpoint_epochs = 5
filename = "v3"

epoch = 1
while epoch <= epochs:
  optimizer.step(lambda: 
                 pinn.closure(wing_df,
                              optimizer, 
                              Nf, N0, Nb, Nw, 
                              x_max, y_max, z_max, t_max, 
                              c1, c2, c3, c4, 
                              in_velocity, 
                              mu, 
                              rho, 
                              device))

  # utils.save_log(log_filepath, epoch, epoch_loss)
  # utils.print_log(epoch, epoch_loss)

  # if np.isnan(epoch_loss[0]):
  #   print("=> NaN loss...")
  #   if epoch % checkpoint_epochs == 0:
  #     pinn, optimizer, checkpoint_epoch = utils.load_checkpoint(pinn, optimizer, models_out_dir + filename + "_" + str(epoch - checkpoint_epochs) + ".pt")
  #   else:
  #     pinn, optimizer, checkpoint_epoch = utils.load_checkpoint(pinn, optimizer, models_out_dir + filename + "_" + str(epoch - (epoch % checkpoint_epochs)) + ".pt")
  #   epoch = checkpoint_epoch + 1
  #   continue

  if epoch % checkpoint_epochs == 0:
    utils.save_checkpoint(pinn, epoch, optimizer, models_out_dir + filename + "_" + str(epoch) + ".pt")

  print("Epoch: ", epoch)

  epoch += 1

KeyboardInterrupt: 

In [ ]:
pinn, optimizer, checkpoint_epoch = utils.load_checkpoint(pinn, optimizer, models_out_dir + filename + "_" + str(epoch - (epoch % checkpoint_epochs)) + ".pt")
# for param_group in optimizer.param_groups:
#     param_group['lr'] = 0.01

# for param_group in optimizer.param_groups:
#   print(param_group['lr'])  # Should print the new learning rate